<div align=\"center\"><h1> Project Pricing Autocall SG </h1></div>

## Import 

### Import libraries

In [1]:
import time
import datetime
from math import sqrt, exp, log
from random import gauss
import scipy.stats
import pandas as pd

In [2]:
# definition of function

def generate_asset_price(S_t,v,r,div,t1,t2):
    # S is spot price
    # V is implied vol
    T= (t2- t1).days/365.0
    return S_t * exp((r - div - 0.5 * v**2) * T + v * sqrt(T) * gauss(0,1.0))

def df(r,t1,t2):
    # function of discount factor between date t1 and t2 with interest rate r
    return exp(-r * (t2-t1).days/365.0)
    

### Import date

In [3]:
tabDate = pd.read_csv('date.csv', sep= ";", infer_datetime_format= True)

In [4]:
date= [datetime.datetime.strptime(x, '%d/%m/%Y') for x in list(tabDate.date)]

## Monte Carlo Simulation

In [5]:
S0 =100 # asset price at the start date
v= 0.20 # Implied volatility
r = 0.04 # taux d'interet
div = 0.00 # dividend rate
N = 1000 # initial investment

l = len(date)
asset_prices = [S0] * l
coupon_barrier = 0.8
kickout_barrier = 1.1
protection_barrier = 0.6
coupon_rate = 0.088
autocall_prices= []

In [6]:
def mcSimulation(simulations):
    start_date = date[0]
    end_date = date[-1]
    for i in range(simulations):
        autocall_price = 0
        checker = True
        payoffs = [0] * l # on remet tous les payoffs a zero
        #Le payoff a la start_date vaut toujours zero

        for j in range(1, l- 1) :
            asset_prices[j]= generate_asset_price(asset_prices[j-1],v,r,div,date[j-1],date[j])
            if asset_prices[j]/S0 < coupon_barrier:
                payoffs[j+1] = payoffs[j]+ N * coupon_rate
                payoffs[j] = 0
                autocall_price += payoffs[j] * df(r,start_date,date[j])
            elif asset_prices[j]/S0 < kickout_barrier:
                payoffs[j] += N * coupon_rate
                autocall_price += payoffs[j]  * df(r,start_date,date[j])
            else:
                payoffs[j] += N * coupon_rate + N
                autocall_price += payoffs[j] * df(r,start_date,date[j])
                checker = False
                break

        if checker: 
            asset_prices[-1]= generate_asset_price(asset_prices[-2],v,r,div,date[-2],date[-1])
            if asset_prices[-1]/S0 < protection_barrier:
                payoffs[-1] = N * asset_prices[l -1]/S0
            elif asset_prices[-1]/S0 < coupon_barrier: 
                payoffs[-1] = N
            else:
                payoffs[-1] += N * coupon_rate + N
        autocall_price += payoffs[-1] * df(r,start_date,end_date)

        autocall_prices.append(autocall_price)

    final_autocall_price = sum(autocall_prices)/float(simulations)
    return final_autocall_price

In [7]:
mcSimulation(1000000)

1050.3679815994055